In [1]:
import pandas as pd
import numpy as np

In [2]:
file=r'C:\Users\akhil\Downloads\train.csv'

In [3]:
data=pd.read_csv(file)

In [4]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
#Treating the missing values
data.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive

In [9]:
#Used to treat missing vales for numeric and categorical
def missing(cat_var,num_var,du):
    du=du.copy()
    for j in num_var:
        z=round(du[j].mean(),1)
        du[j].fillna(z,inplace=True)
    for i in cat_var:
        y=du[i].value_counts().index[0]
        du[i].fillna(y,inplace=True)
    return du 

In [6]:

cat_var=[i for i in data.columns if data[i].dtypes=='O']
num_var_data=[i for i in data.columns if data[i].dtypes!='O']


In [112]:
numeric_without_zero=[]
for i in num_var_data:
    if df[df[i]==0][i].empty==True:
        numeric_without_zero.append(i)
    
    

    


In [113]:
numeric_without_zero

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 '1stFlrSF',
 'GrLivArea',
 'TotRmsAbvGrd',
 'GarageYrBlt',
 'MoSold',
 'YrSold',
 'SalePrice']

In [114]:
#log transform the numeric variables
for i in numeric_without_zero:
    df[i]=np.log(df[i])

In [117]:
# let's explore the relationship between the year variables and the house price in a bit of more details

def elapsed_years(df, var):
    # capture difference between year variable and year the house was sold
    df[var] = df['YrSold'] - df[var]
    return df

In [118]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    df = elapsed_years(df, var)

In [120]:
df.shape

(1460, 81)

In [121]:
dft=df.copy()

In [125]:
#Replaceing the least frequent value with Rare
def find_frequent_labels(df, var, rare_perc):
    # finds the labels that are shared by more than a certain % of the houses in the dataset
    df = df.copy()
    tmp = df.groupby(var)['SalePrice'].count() / len(df)
    return tmp[tmp>rare_perc].index

for var in cat_var:
    frequent_ls = find_frequent_labels(df, var, 0.01)
    df[var] = np.where(df[var].isin(frequent_ls), df[var], 'Rare')
    

In [126]:
df.groupby('MSZoning')['SalePrice'].count()/len(df)

MSZoning
FV      0.044521
RH      0.010959
RL      0.788356
RM      0.149315
Rare    0.006849
Name: SalePrice, dtype: float64

In [122]:
cat_var

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [132]:
# this function will assign discrete values to the strings of the variables, 
# so that the smaller value corresponds to the smaller mean of target

def replace_categories(df,var):
    ordered_labels = df.groupby([var])['SalePrice'].mean().sort_values().index
    ordinal_label = {k:i for i, k in enumerate(ordered_labels, 0)} 
    df[var] = df[var].map(ordinal_label)
    
    

In [130]:
df.groupby(['MSZoning'])['SalePrice'].mean().sort_values()

MSZoning
Rare    11.118259
RM      11.692893
RH      11.749840
RL      12.085885
FV      12.246616
Name: SalePrice, dtype: float64

In [133]:
for i in cat_var:
    replace_categories(df,i)

In [134]:
df[cat_var]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,1,1,0,1,1,0,0,14,2,...,3,1,1,2,2,0,2,1,2,3
1,3,1,1,0,1,1,2,0,11,1,...,3,1,1,2,2,0,2,1,2,3
2,3,1,1,1,1,1,0,0,14,2,...,3,1,1,2,2,0,2,1,2,3
3,3,1,1,1,1,1,1,0,16,2,...,1,0,1,2,2,0,2,1,2,0
4,3,1,1,1,1,1,2,0,22,2,...,3,1,1,2,2,0,2,1,2,3
5,3,1,1,1,1,1,0,0,9,2,...,3,0,1,2,2,0,2,1,2,3
6,3,1,1,0,1,1,0,0,18,2,...,3,1,1,2,2,0,2,1,2,3
7,3,1,1,1,1,1,1,0,12,5,...,3,1,1,2,2,0,2,1,2,3
8,1,1,1,0,1,1,0,0,4,0,...,1,0,0,2,2,0,2,1,2,0
9,3,1,1,0,1,1,1,0,3,0,...,3,1,2,2,2,0,2,1,2,3


In [136]:
train_vars = [var for var in df.columns if var not in ['Id', 'SalePrice']]
len(train_vars)

79

In [138]:
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

In [139]:
scaler=MinMaxScaler()

In [140]:
# fit scaler
scaler = MinMaxScaler() # create an instance
scaler.fit(df[train_vars]) #  fit  the scaler to the train set for later use



A:\py\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [141]:
# transform the train and test set, and add on the Id and SalePrice variables
df = pd.concat([df[['Id', 'SalePrice']].reset_index(drop=True),
                    pd.DataFrame(scaler.transform(df[train_vars]), columns=train_vars)],
                    axis=1)

In [143]:
df.shape

(1460, 81)

In [144]:
x_train=df.drop(['Id','SalePrice'],1)
y_train=df['SalePrice']

In [145]:
# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [146]:
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=0)) # remember to set the seed, the random state in this function
sel_.fit(x_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)

In [147]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feat = x_train.columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((x_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

total features: 79
selected features: 19
features with coefficients shrank to zero: 60


In [149]:
[selected_feat]

,MSSubClass,Neighborhood,OverallQual,YearRemodAdd,RoofStyle,ExterQual,BsmtQual,BsmtExposure,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,KitchenQual,Fireplaces,GarageFinish,GarageCars,PavedDrive,SaleCondition
0,0.487992,0.636364,0.845098,0.097108,0.0,0.666667,0.666667,0.000000,1.00,1.0,0.356155,0.577712,0.333333,0.666667,0.000000,0.5,0.50,1.0,0.75
1,0.000000,0.500000,0.778151,0.521519,0.0,0.333333,0.666667,1.000000,1.00,1.0,0.503056,0.470245,0.000000,0.333333,0.333333,0.5,0.50,1.0,0.75
2,0.487992,0.636364,0.845098,0.113320,0.0,0.666667,0.666667,0.333333,1.00,1.0,0.383441,0.593095,0.333333,0.666667,0.333333,0.5,0.50,1.0,0.75
3,0.556464,0.727273,0.845098,0.604064,0.0,0.333333,0.333333,0.000000,0.75,1.0,0.399941,0.579157,0.333333,0.666667,0.333333,0.0,0.75,1.0,0.00
4,0.487992,1.000000,0.903090,0.145768,0.0,0.666667,0.666667,0.666667,1.00,1.0,0.466237,0.666523,0.333333,0.666667,0.333333,0.5,0.75,1.0,0.75
5,0.407007,0.409091,0.698970,0.243192,0.0,0.333333,0.666667,0.000000,1.00,1.0,0.328654,0.497220,0.333333,0.333333,0.000000,0.0,0.50,1.0,0.75
6,0.000000,0.818182,0.903090,0.048538,0.0,0.666667,1.000000,0.666667,1.00,1.0,0.614465,0.574387,0.333333,0.666667,0.333333,0.5,0.50,1.0,0.75
7,0.487992,0.545455,0.845098,0.603178,0.0,0.333333,0.666667,0.333333,1.00,1.0,0.453465,0.648700,0.333333,0.333333,0.666667,0.5,0.50,1.0,0.75
8,0.407007,0.181818,0.845098,0.967682,0.0,0.333333,0.333333,0.000000,0.75,1.0,0.423231,0.590710,0.000000,0.333333,0.666667,0.0,0.50,1.0,0.00
9,1.000000,0.136364,0.698970,0.967682,0.0,0.333333,0.333333,0.000000,1.00,1.0,0.443067,0.414169,0.333333,0.333333,0.666667,0.5,0.25,1.0,0.75


Index(['MSSubClass', 'Neighborhood', 'OverallQual', 'YearRemodAdd',
       'RoofStyle', 'ExterQual', 'BsmtQual', 'BsmtExposure', 'HeatingQC',
       'CentralAir', '1stFlrSF', 'GrLivArea', 'BsmtFullBath', 'KitchenQual',
       'Fireplaces', 'GarageFinish', 'GarageCars', 'PavedDrive',
       'SaleCondition'],
      dtype='object')

In [151]:
x_tr,x_tst=

,MSSubClass,Neighborhood,OverallQual,YearRemodAdd,RoofStyle,ExterQual,BsmtQual,BsmtExposure,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,KitchenQual,Fireplaces,GarageFinish,GarageCars,PavedDrive,SaleCondition
0,0.487992,0.636364,0.845098,0.097108,0.0,0.666667,0.666667,0.000000,1.00,1.0,0.356155,0.577712,0.333333,0.666667,0.000000,0.5,0.50,1.0,0.75
1,0.000000,0.500000,0.778151,0.521519,0.0,0.333333,0.666667,1.000000,1.00,1.0,0.503056,0.470245,0.000000,0.333333,0.333333,0.5,0.50,1.0,0.75
2,0.487992,0.636364,0.845098,0.113320,0.0,0.666667,0.666667,0.333333,1.00,1.0,0.383441,0.593095,0.333333,0.666667,0.333333,0.5,0.50,1.0,0.75
3,0.556464,0.727273,0.845098,0.604064,0.0,0.333333,0.333333,0.000000,0.75,1.0,0.399941,0.579157,0.333333,0.666667,0.333333,0.0,0.75,1.0,0.00
4,0.487992,1.000000,0.903090,0.145768,0.0,0.666667,0.666667,0.666667,1.00,1.0,0.466237,0.666523,0.333333,0.666667,0.333333,0.5,0.75,1.0,0.75
5,0.407007,0.409091,0.698970,0.243192,0.0,0.333333,0.666667,0.000000,1.00,1.0,0.328654,0.497220,0.333333,0.333333,0.000000,0.0,0.50,1.0,0.75
6,0.000000,0.818182,0.903090,0.048538,0.0,0.666667,1.000000,0.666667,1.00,1.0,0.614465,0.574387,0.333333,0.666667,0.333333,0.5,0.50,1.0,0.75
7,0.487992,0.545455,0.845098,0.603178,0.0,0.333333,0.666667,0.333333,1.00,1.0,0.453465,0.648700,0.333333,0.333333,0.666667,0.5,0.50,1.0,0.75
8,0.407007,0.181818,0.845098,0.967682,0.0,0.333333,0.333333,0.000000,0.75,1.0,0.423231,0.590710,0.000000,0.333333,0.666667,0.0,0.50,1.0,0.00
9,1.000000,0.136364,0.698970,0.967682,0.0,0.333333,0.333333,0.000000,1.00,1.0,0.443067,0.414169,0.333333,0.333333,0.666667,0.5,0.25,1.0,0.75


In [160]:
# transform the train and test set, and add on the Id and SalePrice variables
df1 = pd.concat([df['SalePrice'].reset_index(drop=True),
                    df[selected_feat]],
                    axis=1)

In [162]:
from sklearn.model_selection import train_test_split

In [163]:
x_tr,x_tst=train_test_split(df1,test_size=0.2,random_state=2)

In [164]:
x_tra=x_tr.drop(['SalePrice'],1)
y_tra=x_tr['SalePrice']
x_test=x_tst.drop(['SalePrice'],1)
y_test=x_tst['SalePrice']

In [165]:
from sklearn.linear_model import LinearRegression

In [166]:
lm=LinearRegression()

In [171]:
lm.fit(x_tr,y_tra)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [170]:
x_tra.reset_index(drop=True,inplace=True)
y_tra.reset_index(drop=True,inplace=True)
x_test.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

In [174]:
x_tra.shape

(1168, 19)

In [175]:
x_test.shape

(292, 19)

In [181]:
vf=[i for i in df.columns if i not in selected_feat]

In [183]:
vf.remove('SalePrice')

In [185]:
len(vf)

61

In [186]:
df.drop(vf,axis=1,inplace=True)

In [190]:
train,test=train_test_split(df,test_size=0.2,random_state=2)

In [191]:
X_train=train.drop(['SalePrice'],1)

In [192]:
Y_train=train['SalePrice']

In [193]:
X_test=test.drop(['SalePrice'],1)

In [194]:
Y_test=test['SalePrice']

In [198]:
Y_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)
Y_test.reset_index(drop=True,inplace=True)

In [199]:
lm1=LinearRegression()

In [200]:
lm1.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [201]:
pred=lm1.predict(X_test)

In [203]:
rsid=pred-Y_test

In [204]:
#RMSE is very low hence our model performance is great
np.sqrt(np.dot(rsid,rsid)/len(pred))

0.13649640037981323

In [ ]:
y